In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import numpy as np
import cv2
from PIL import Image
from skimage.measure import regionprops, label, find_contours
from scipy.spatial.distance import cdist

import pandas as pd

In [ ]:
#値を-1から1に正規化する関数
def normalize_x(image):
    return image / 127.5 - 1


def denormalize_x(image):
    return (image + 1) * 127.5


#値を0から1正規化する関数
def normalize_y(image):
    return image / 255


#値を0から255に戻す関数
def denormalize_y(image):
    return image * 255

In [ ]:
# インプット画像を読み込む関数
def load_X_gray(folder_path):
    
    image_files = []

    #image_files = os.listdir(folder_path)
       
    for file in os.listdir(folder_path):
        base, ext = os.path.splitext(file)
        if ext == '.png':
            image_files.append(file)
        else :
            pass
        
    image_files.sort()
    print (image_files)
    
    img = cv2.imread(folder_path + os.sep + image_files[0], cv2.IMREAD_GRAYSCALE)
    
    #image_files = image_files[1:]
    images = np.zeros((len(image_files), img.shape[0], img.shape[1], 1), np.float32)
    for i, image_file in enumerate(image_files):
        image = cv2.imread(folder_path + os.sep + image_file, cv2.IMREAD_GRAYSCALE)
        print (image.shape)
        #image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
        image = image[:, :, np.newaxis]
        images[i] = normalize_x(image)
    return images, image_files


def load_Y_gray_with_gaussian_and_numeric(folder_path, thresh = None , normalize = True, g_size = None):
    image_files = []
    #image_files = os.listdir(folder_path)
    
    for file in os.listdir(folder_path):
        base, ext = os.path.splitext(file)
        if ext == '.png':
            image_files.append(int(base))
        else:
            pass
        
    image_files.sort()
    print(image_files)
    
    img = cv2.imread(folder_path + os.sep + str(image_files[0]).zfill(4) + ".png", cv2.IMREAD_GRAYSCALE)
    
    images = np.zeros(
        (len(image_files), img.shape[0], img.shape[1], 1) ,np.float32
    )
    
    for i , image_file in enumerate(image_files):
        image = cv2.imread(
            folder_path + os.sep + str(image_file).zfill(4) + ".png" ,
            cv2.IMREAD_GRAYSCALE
        )
        #print(image.shape)
        
        # ぼかし処理
        if g_size:
            image = cv2.GaussianBlur(
                image, (g_size, g_size), 0
            )
        
        
        
        if thresh:
            ret , image = cv2.threshold(image , thresh , 255 , cv2.THRESH_BINARY)
        image = image[ : , : , np.newaxis]
        if normalize:
            images[i] = normalize_y(image)
        else:
            images[i] = image
            
    print(images.shape)
    
    return images , image_files

In [ ]:
def make_mask_img(ori_img, mask_img):
    mask_img_rgb = np.zeros((mask_img.shape[0], mask_img.shape[1], 3), np.float32)
    mask_img_rgb[:,:,0] = mask_img[:,:,0]
    mask_img_rgb[:,:,2] = mask_img[:,:,0]
    masked_img = cv2.addWeighted(mask_img_rgb,0.5,cv2.cvtColor(ori_img+0.75, cv2.COLOR_GRAY2BGR),0.6,0)
    return masked_img

In [ ]:
def kakudai_img_500(img):
    kakudai_img = np.zeros((500, 500, 3), np.float32)
    
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            kakudai_img[i*25:(i+1)*25, j*25:(j+1)*25,:] = img[i, j, :]
     
    return kakudai_img

In [ ]:
ori_imgs, _ = load_X_gray("Z:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/tap_dataset/cropped_0_20/original//")
baseline_imgs, _ = load_Y_gray_with_gaussian_and_numeric("Z:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/tap_dataset/cropped_0_20/baseline/", normalize=False)
tap_img, _ =  load_Y_gray_with_gaussian_and_numeric("Z:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/tap_dataset/cropped_0_20/tap//", normalize=False)


In [ ]:
out_dir = "Z:/paper_data/2021_cristae/for_revision_#2/figS1A/"

os.makedirs(f"{out_dir}/gt", exist_ok  = True)
os.makedirs(f"{out_dir}/baseline", exist_ok  = True)

cv2.imwrite(
    f"{out_dir}/gt/xy.png",
    kakudai_img_500(denormalize_y(
        make_mask_img(ori_imgs[4][100:120, 90:110], tap_img[4][100:120, 90:110])
    )
))

cv2.imwrite(
    f"{out_dir}/baseline/xy.png",
    kakudai_img_500(denormalize_y(
        make_mask_img(ori_imgs[4][100:120, 90:110], baseline_imgs[4][100:120, 90:110])
    )
))

cv2.imwrite(
    f"{out_dir}/gt/yz.png",
    kakudai_img_500(denormalize_y(
        make_mask_img(ori_imgs.transpose(2, 0, 1, 3)[100][:, 100:120], tap_img.transpose(2, 0, 1, 3)[100][:, 100:120])
    )
))

cv2.imwrite(
    f"{out_dir}/baseline/yz.png",
    kakudai_img_500(denormalize_y(
        make_mask_img(ori_imgs.transpose(2, 0, 1, 3)[100][:, 100:120], baseline_imgs.transpose(2, 0, 1, 3)[100][:, 100:120])
    )
))

cv2.imwrite(
    f"{out_dir}/gt/zx.png",
    kakudai_img_500(denormalize_y(
        make_mask_img(ori_imgs.transpose(1, 2, 0, 3)[128][90:110, :], tap_img.transpose(1, 2, 0, 3)[128][90:110, :])
    )
))

cv2.imwrite(
    f"{out_dir}/baseline/zx.png",
    kakudai_img_500(denormalize_y(
        make_mask_img(ori_imgs.transpose(1, 2, 0, 3)[128][90:110, :], baseline_imgs.transpose(1, 2, 0, 3)[128][90:110, :])
    )
))